In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
3,US might not get back to pre-pandemic life unt...,https://www.cnn.com/2020/09/11/health/us-coron...,"Fri, 11 Sep 2020 21:42:00 GMT",2020-09-11 21:42:00+00:00
1,COVID-19 worries douse plans for fire experime...,https://www.sciencemag.org/news/2020/09/covid-...,"Fri, 11 Sep 2020 18:30:00 GMT",2020-09-11 18:30:00+00:00
4,Covid-19 News: Live Updates The New York Times,https://www.nytimes.com/2020/09/11/world/covid...,"Fri, 11 Sep 2020 17:32:51 GMT",2020-09-11 17:32:51+00:00
5,Governor Cuomo Announces 35th Straight Day wit...,https://www.governor.ny.gov/news/governor-cuom...,"Fri, 11 Sep 2020 16:39:03 GMT",2020-09-11 16:39:03+00:00
0,COVID-19 Daily Update 9-11-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Fri, 11 Sep 2020 14:10:34 GMT",2020-09-11 14:10:34+00:00
7,Texas' COVID-19 positivity rate is falling Th...,https://www.texastribune.org/2020/09/11/texas-...,"Fri, 11 Sep 2020 10:00:00 GMT",2020-09-11 10:00:00+00:00
8,The Other Way Covid Will Kill: Hunger The New...,https://www.nytimes.com/2020/09/11/business/co...,"Fri, 11 Sep 2020 09:00:00 GMT",2020-09-11 09:00:00+00:00
2,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Thu, 10 Sep 2020 21:00:00 GMT",2020-09-10 21:00:00+00:00
9,Governor Cuomo Announces Over 9 Million COVID-...,https://www.governor.ny.gov/news/governor-cuom...,"Thu, 10 Sep 2020 17:14:33 GMT",2020-09-10 17:14:33+00:00
6,NIH ACTIV initiative launches adaptive clinica...,https://www.nih.gov/news-events/news-releases/...,"Wed, 09 Sep 2020 15:25:29 GMT",2020-09-09 15:25:29+00:00
